In [1]:
import pandas as pd
import numpy as np

np.random.seed(123)

In [2]:
students = ['Sally', 'Jane', 'Suzie', 'Billy', 'Ada', 'John', 'Thomas',
            'Marie', 'Albert', 'Richard', 'Isaac', 'Alan']

# randomly generate scores for each student for each subject
# note that all the values need to have the same length here
math_grades = np.random.randint(low=60, high=100, size=len(students))
english_grades = np.random.randint(low=60, high=100, size=len(students))
reading_grades = np.random.randint(low=60, high=100, size=len(students))

df = pd.DataFrame({'name': students,
                   'math': math_grades,
                   'english': english_grades,
                   'reading': reading_grades,
                   'classroom': np.random.choice(['A', 'B'], len(students))})

In [3]:
# What does each row/observation represent?
# an individual student record of their grades
# A student and their grades and their classroom
df.head()

,name,math,english,reading,classroom
0,Sally,62,85,80,A
1,Jane,88,79,67,B
2,Suzie,94,74,95,A
3,Billy,98,96,88,B
4,Ada,77,92,98,A


## When looking at our data
- Always ask, what does each row/observation represent?

## Handy Pandas Functions for Reading Data
- `pd.read_csv("filename.csv")`
- `pd.read_excel("filename.xlsx")`
- `pd.read_json("filename.json")`
- `pd.read_sql` uses a syntatically correct SQL query and a connection object
- `pd.read_table` can read a HTML table of content 
- `pd.read_clipboard()` can read your clipboard into a dataframe

# Let's Have Pandas and SQL Talk! Together at last!

In [6]:
from env import host, user, password

# Connection string
url = f'mysql+pymysql://{user}:{password}@{host}/employees'
    
query = """
    SELECT * FROM employees limit 5
"""

pd.read_sql(query, url)

,emp_no,birth_date,first_name,last_name,gender,hire_date
0,10001,1953-09-02,Georgi,Facello,M,1986-06-26
1,10002,1964-06-02,Bezalel,Simmel,F,1985-11-21
2,10003,1959-12-03,Parto,Bamford,M,1986-08-28
3,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01
4,10005,1955-01-21,Kyoichi,Maliniak,M,1989-09-12


## Let's Talk Aggregation
- Aggregation and grouping by allow us to put stuff together.
- Can consider many things, one thing
- Changes our dimension of observation
- Literally changes what each row in a df means

In [7]:
df.head()

,name,math,english,reading,classroom
0,Sally,62,85,80,A
1,Jane,88,79,67,B
2,Suzie,94,74,95,A
3,Billy,98,96,88,B
4,Ada,77,92,98,A


In [14]:
# Remember double brackets mean a dataframe
# we get a dataframe here w/ reading, english, math, and we lose a level of detail
# We lose names and classrooms
# Aggregating changes what each row means
df[['english', 'reading', 'math']].agg(['mean', 'median', 'min', 'max', 'count'])

,english,reading,math
mean,77.666667,86.5,84.833333
median,77.500000,89.0,90.000000
min,62.000000,67.0,62.000000
max,99.000000,98.0,98.000000
count,12.000000,12.0,12.000000


In [12]:
# Show the English average
df.english.mean()

77.66666666666667

In [13]:
# Show the median math grade
df.math.median()

90.0

In [17]:
df.head()

,name,math,english,reading,classroom
0,Sally,62,85,80,A
1,Jane,88,79,67,B
2,Suzie,94,74,95,A
3,Billy,98,96,88,B
4,Ada,77,92,98,A


In [20]:
# Grouping By means we're grouping our original rows together by some column's value
# To Group By:
# Step 1 is we group by a column
# Step 2 is we need an aggregate function
# Take stock of how we're changing our observation/rows
# from a single student to a group of rows by classroom letter
# Average math score by classroom.
df.groupby("classroom").math.mean()

classroom
A    82.625
B    89.250
Name: math, dtype: float64

In [21]:
# Grouping by creates a "group by object"
df.groupby("classroom").math.agg(["mean", "median"])

,mean,median
classroom,,
A,82.625,87
B,89.250,90


In [22]:
# Think of the SQL that might correspond to this:
# select classroom, count 
# from students
# group by classroom
df.groupby("classroom").name.count()

classroom
A    8
B    4
Name: name, dtype: int64

In [23]:
df.groupby("classroom").describe()

math                                                    english  \
          count    mean        std   min    25%   50%    75%   max   count   
classroom                                                                    
A           8.0  82.625  12.281665  62.0  75.00  87.0  92.25  94.0     8.0   
B           4.0  89.250   7.973916  79.0  85.75  90.0  93.50  98.0     4.0   

                  ...              reading                                  \
            mean  ...    75%   max   count    mean        std   min    25%   
classroom         ...                                                        
A          72.75  ...  81.25  92.0     8.0  87.125   8.887190  72.0  80.75   
B          87.50  ...  96.75  99.0     4.0  85.250  12.392874  67.0  82.75   

                              
            50%    75%   max  
classroom                     
A          88.5  94.25  98.0  
B          90.5  93.00  93.0  

[2 rows x 24 columns]

## Takeaways:
- Whatever column/value we "group by" becomes the new observation level
- If the original rows were students, each observation is a student
- when we groupby the classroom, each row/observation is a classroom


In [26]:
# Make a "passing_math" column
df["passing_math"] = df.math >= 70
df.head()

,name,math,english,reading,classroom,passing_math
0,Sally,62,85,80,A,False
1,Jane,88,79,67,B,True
2,Suzie,94,74,95,A,True
3,Billy,98,96,88,B,True
4,Ada,77,92,98,A,True


In [28]:
df.groupby(["classroom", "passing_math"]).reading.agg(["mean", "median", "min", "max"])

mean  median  min  max
classroom passing_math                             
A         False         87.000000    87.0   80   94
          True          87.166667    88.5   72   98
B         True          85.250000    90.5   67   93

In [30]:
df.groupby(["classroom", "passing_math"]).name.count()

classroom  passing_math
A          False           2
           True            6
B          True            4
Name: name, dtype: int64

In [32]:
df.groupby(["classroom", "passing_math"]).math.agg(["count", "mean"])

count       mean
classroom passing_math                  
A         False             2  65.500000
          True              6  88.333333
B         True              4  89.250000

In [33]:
# .transform in the way below allows us to calculate an aggregate and append that aggregate onto the original dataframe
df.assign(avg_math_score_by_classroom=df.groupby('classroom').math.transform('mean'))

,name,math,english,reading,classroom,passing_math,avg_math_score_by_classroom
0,Sally,62,85,80,A,False,82.625
1,Jane,88,79,67,B,True,89.250
2,Suzie,94,74,95,A,True,82.625
3,Billy,98,96,88,B,True,89.250
4,Ada,77,92,98,A,True,82.625
5,John,79,76,93,B,True,89.250
6,Thomas,82,64,81,A,True,82.625
7,Marie,93,63,90,A,True,82.625
8,Albert,92,62,87,A,True,82.625
9,Richard,69,80,94,A,False,82.625


In [37]:
# Aggregation example:
# Let's count the number of employees in each department from the employees_db

# Replicate the SQL here w/ pandas groupby and aggregation
# select dept_name, count(*)
# from departments
# join dept_emp using(dept_no)
# where to_date > now()
# group by dept_name;

sql = """
select *
from departments
join dept_emp using(dept_no)
where to_date > now()
"""

employees = pd.read_sql(sql, url)
employees.head()

,dept_no,dept_name,emp_no,from_date,to_date
0,d009,Customer Service,10038,1989-09-20,9999-01-01
1,d009,Customer Service,10049,1992-05-04,9999-01-01
2,d009,Customer Service,10060,1992-11-11,9999-01-01
3,d009,Customer Service,10088,1992-03-21,9999-01-01
4,d009,Customer Service,10112,1998-05-01,9999-01-01


In [39]:
employees.groupby("dept_name").emp_no.count()

dept_name
Customer Service      17569
Development           61386
Finance               12437
Human Resources       12898
Marketing             14842
Production            53304
Quality Management    14546
Research              15441
Sales                 37701
Name: emp_no, dtype: int64

## In the code example above, what does each row represent?
- Always ask what each row/observation means, especially when aggregating

In [40]:
# Let's get minimum, mean, median, and maximum salary numbers by gender
sql = """
select *
from employees
join salaries using(emp_no)
where to_date > now();
"""

employees = pd.read_sql(sql, url)
employees.head()

,emp_no,birth_date,first_name,last_name,gender,hire_date,salary,from_date,to_date
0,10001,1953-09-02,Georgi,Facello,M,1986-06-26,88958,2002-06-22,9999-01-01
1,10002,1964-06-02,Bezalel,Simmel,F,1985-11-21,72527,2001-08-02,9999-01-01
2,10003,1959-12-03,Parto,Bamford,M,1986-08-28,43311,2001-12-01,9999-01-01
3,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01,74057,2001-11-27,9999-01-01
4,10005,1955-01-21,Kyoichi,Maliniak,M,1989-09-12,94692,2001-09-09,9999-01-01


In [41]:
employees.groupby("gender").salary.agg(["min", "mean", "median", "max", "count"])

,min,mean,median,max,count
gender,,,,,
F,38936,71963.570753,69764,152710,96010
M,38623,72044.656973,69830,158220,144114


In [43]:
# Do the same as above, but group by dept_name AND gender
sql = """
select *
from employees
join salaries using(emp_no)
join dept_emp using(emp_no)
join departments using(dept_no)
where dept_emp.to_date > now()
and salaries.to_date > now()
"""

employees = pd.read_sql(sql, url)
employees.head()

,dept_no,emp_no,birth_date,first_name,last_name,gender,hire_date,salary,from_date,to_date,from_date,to_date,dept_name
0,d009,10038,1960-07-20,Huan,Lortz,M,1989-09-20,64254,2001-09-17,9999-01-01,1989-09-20,9999-01-01,Customer Service
1,d009,10049,1961-04-24,Basil,Tramer,F,1992-05-04,51326,2002-05-02,9999-01-01,1992-05-04,9999-01-01,Customer Service
2,d009,10060,1961-10-15,Breannda,Billingsley,M,1987-11-02,93188,2002-05-25,9999-01-01,1992-11-11,9999-01-01,Customer Service
3,d009,10088,1954-02-25,Jungsoon,Syrzycki,F,1988-09-02,98003,2001-08-30,9999-01-01,1992-03-21,9999-01-01,Customer Service
4,d009,10112,1963-08-13,Yuichiro,Swick,F,1985-10-08,61070,2002-04-30,9999-01-01,1998-05-01,9999-01-01,Customer Service


In [44]:
employees.groupby(["dept_name", "gender"]).salary.agg(["min", "mean", "median", "max", "count"])

min          mean   median     max  count
dept_name          gender                                             
Customer Service   F       39812  67409.493935  65198.0  144866   7007
                   M       39373  67202.791611  65100.0  143950  10562
Development        F       39469  67575.845881  66355.0  144434  24533
                   M       39036  67712.555911  66526.0  140784  36853
Finance            F       39949  78747.416633  78285.0  136978   5014
                   M       39012  78433.300418  77526.0  142395   7423
Human Resources    F       38936  64139.954731  62782.0  123268   5147
                   M       39611  63777.101922  62864.0  141953   7751
Marketing          F       39871  79699.771487  78595.5  141842   5864
                   M       39821  80293.380931  79481.0  145128   8978
Production         F       39476  67728.112560  66645.0  138273  21393
                   M       38623  67920.524490  66768.0  132552  31911
Quality Management F       39571  65561.837704  64685.0  122965   5872
                   M       38942  65360.862924  64258.0  132103   8674
Research           F       39526  68011.855201  66814.0  124158   6181
                   M       39186  67847.639957  66576.5  130211   9260
Sales              F       40392  88835.963931  88581.0  152710  14999
                   M       39426  88864.204872  88461.5  158220  22702

In [47]:
# Crosstab and .pivot_table
pd.crosstab(df.passing_math, df.classroom)

classroom,A,B
passing_math,,
False,2,0
True,6,4


In [53]:
pd.crosstab(employees.gender, employees.dept_name)

dept_name,Customer Service,Development,Finance,Human Resources,Marketing,Production,Quality Management,Research,Sales
gender,,,,,,,,,
F,7007,24533,5014,5147,5864,21393,5872,6181,14999
M,10562,36853,7423,7751,8978,31911,8674,9260,22702


In [54]:
pd.crosstab(employees.gender, employees.dept_name, normalize=True)

dept_name,Customer Service,Development,Finance,Human Resources,Marketing,Production,Quality Management,Research,Sales
gender,,,,,,,,,
F,0.029181,0.102168,0.020881,0.021435,0.024421,0.089091,0.024454,0.025741,0.062464
M,0.043986,0.153475,0.030913,0.032279,0.037389,0.132894,0.036123,0.038563,0.094543


In [56]:
pd.crosstab(employees.gender, employees.dept_name, margins=True)

dept_name,Customer Service,Development,Finance,Human Resources,Marketing,Production,Quality Management,Research,Sales,All
gender,,,,,,,,,,
F,7007,24533,5014,5147,5864,21393,5872,6181,14999,96010
M,10562,36853,7423,7751,8978,31911,8674,9260,22702,144114
All,17569,61386,12437,12898,14842,53304,14546,15441,37701,240124


In [57]:
len([1, 2, 3])

3

In [58]:
len

<function len(obj, /)>

In [60]:
def nudge_grades(df):
    # cranks up the math score by 10%
    # cranks down the reading by 10%
    df.math = df.math + df.math * .10
    df.reading = df.reading - df.reading * .10
    return df

In [61]:
def bump_english(df):
    df.english = df.english + 5
    return df

In [64]:
# With nested function calls like this, the order of operations is from the inside to the outside
# which kinda feels like reading from right to left
bump_english(nudge_grades(df)).head()

,name,math,english,reading,classroom,passing_math
0,Sally,82.522,95,58.320,A,False
1,Jane,117.128,89,48.843,B,True
2,Suzie,125.114,84,69.255,A,True
3,Billy,130.438,106,64.152,B,True
4,Ada,102.487,102,71.442,A,True


In [72]:
# .pipe exists to take in function definitions and chain/read from left to right
# .pipes allows reading and order of application of the function from left to right.
df.pipe(nudge_grades).pipe(bump_english).head()

,name,math,english,reading,classroom,passing_math
0,Sally,160.812033,130,27.894275,A,False
1,Jane,228.249336,124,23.361455,B,True
2,Suzie,243.811791,119,33.124452,A,True
3,Billy,254.186761,141,30.683703,B,True
4,Ada,199.718169,137,34.170487,A,True


In [74]:
employees.head()

,dept_no,emp_no,birth_date,first_name,last_name,gender,hire_date,salary,from_date,to_date,from_date,to_date,dept_name
0,d009,10038,1960-07-20,Huan,Lortz,M,1989-09-20,64254,2001-09-17,9999-01-01,1989-09-20,9999-01-01,Customer Service
1,d009,10049,1961-04-24,Basil,Tramer,F,1992-05-04,51326,2002-05-02,9999-01-01,1992-05-04,9999-01-01,Customer Service
2,d009,10060,1961-10-15,Breannda,Billingsley,M,1987-11-02,93188,2002-05-25,9999-01-01,1992-11-11,9999-01-01,Customer Service
3,d009,10088,1954-02-25,Jungsoon,Syrzycki,F,1988-09-02,98003,2001-08-30,9999-01-01,1992-03-21,9999-01-01,Customer Service
4,d009,10112,1963-08-13,Yuichiro,Swick,F,1985-10-08,61070,2002-04-30,9999-01-01,1998-05-01,9999-01-01,Customer Service


In [75]:
# .loc[starting_row:ending_row, starting_column:ending_column]
employees.loc[0:3, 'birth_date':'salary']

,birth_date,first_name,last_name,gender,hire_date,salary
0,1960-07-20,Huan,Lortz,M,1989-09-20,64254
1,1961-04-24,Basil,Tramer,F,1992-05-04,51326
2,1961-10-15,Breannda,Billingsley,M,1987-11-02,93188
3,1954-02-25,Jungsoon,Syrzycki,F,1988-09-02,98003
